In [1]:
from PIL import Image
import pandas as pd
import numpy as np

dataRep = '../data/'
datasetRep = '../data/dataset/'

In [2]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [3]:
def makeSquareImage(im, fill_color=(0, 0, 0)):
    x, y = im.size
    size = max(x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(im, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

def convertImage(image, xpixel=500, ypixel=500, gray=False):
    #convert size
    image.thumbnail([xpixel, ypixel], Image.LANCZOS)
    image.convert('RGB')
    image = makeSquareImage(image, fill_color=(255,255,255))
    
    #convert color
    if gray:
        image.convert('L')
        
    return image

def createDataset(xpixel=500, ypixel=500, gray=False):
    trainData = pd.read_csv(dataRep+'train.csv')
    sampleSubData = pd.read_csv(dataRep+'sample_submission.csv')
    nImage = trainData.shape[0] + sampleSubData.shape[0]
    
    i=0
    print('Convert train image')
    print('\n')
    for idImage in trainData['image_id']:
        i+=1
        update_progress(float(i)/float(nImage))
        image = Image.open(dataRep+'train_images/'+idImage+'.jpg')
        image = convertImage(image,xpixel, ypixel, gray)
        image.save(datasetRep+'train/'+idImage+'.jpg')

    print('Convert test image')
    print('\n')
    for idImage in sampleSubData['image_id']:
        i+=1
        update_progress(float(i)/float(nImage))
        image = Image.open(dataRep+'test_images/'+idImage+'.jpg')
        image = convertImage(image,xpixel, ypixel, gray)
        image.save(datasetRep+'submission/'+idImage+'.jpg')
        
        

In [4]:
createDataset()

Progress: [####################] 100.0%
